# Preprocesamiento

En esta etapa generamos las imágenes que utilizará el modelo para el entrenamiento y la predicción. 

Primero se generan las imágenes de n-bandas. Es decir las imágenes que no solo poseen los canales RGB sino además información de otras bandas como el infrarrojo cercano (NIR near-infra-red),  o mismo, una combinación de bandas. En esta etapa consideramos toda la información que podría ser sensible a la detección del objeto de interés.

Luego, utilizando las imágenes generadas en el paso anterior y la información vectorial de la ubicación de los basurales, generamos el dataset utilizado luego por el modelo de ML. Este dataset se genera con el formato necesario para el modelo.



## Preprocesamiento para las imagenes de entrenamiento


Descargamos las librerías glob y os para la navegación de archivos, y tqdm para visualizar el progreso del procesamiento.

In [8]:
GFS_PUBLIC_ROUTE = "dym-datasets-public/basurales/"
DATA_DATE = "2021-02-01_2021-02-28"
IMAGES_OUTPUT = "images_s2/"
BASE_PATH = "/app"
GET_IMAGES = 2

In [2]:
import glob
import os
from tqdm import tqdm

### Descarga de imagenes 

In [3]:
from pathlib import Path


def fullpath(fp):
    return str(Path(f"{BASE_PATH}/{fp}").resolve())

def gfs_route(route):
    return str(Path(f"{GFS_PUBLIC_ROUTE}/{route}"))
        
def mkdir(fp):
    Path(fp).mkdir(parents=True, exist_ok=True)


mkdir(fullpath(f"{IMAGES_OUTPUT}"))
fullpath(IMAGES_OUTPUT)

'/app/images_s2'

In [ ]:
# !mkdir ../images_s2/

In [4]:
gfs_route(f"train_6D/{DATA_DATE}")

'dym-datasets-public/basurales/train_6D/2021-02-01_2021-02-28'

In [6]:
import gcsfs
fs = gcsfs.GCSFileSystem()
fs.ls(gfs_route("train_6D"))

['dym-datasets-public/basurales/train_6D/2021-02-01_2021-02-28']

In [9]:

images_train = fs.ls(gfs_route(f"/train_6D/{DATA_DATE}"))
local_train_path = fullpath(f"{IMAGES_OUTPUT}/train_6D/{DATA_DATE}")
mkdir(local_train_path)
for tiff in images_train[:GET_IMAGES]:
    filename = tiff.rsplit("/", maxsplit=1)[1]
    print(f"Downloading {tiff} into {local_train_path}")
    fs.download(tiff, f"{local_train_path}/{filename}")
    
images_pred = fs.ls(gfs_route(f"/predict_6D/{DATA_DATE}"))
local_pred_path = fullpath(f"{IMAGES_OUTPUT}/predict_6D/{DATA_DATE}")
for tiff in images_pred[:GET_IMAGES]:
    filename = tiff.rsplit("/", maxsplit=1)[1]
    print(f"Downloading {tiff} into {local_pred_path}")
    fs.download(tiff, f"{local_pred_path}/{filename}")

    

In [ ]:
# !gsutil -m cp -r gs://dym-datasets-public/basurales/train_6D/ ../images_s2/
# !gsutil -m cp -r gs://dym-datasets-public/basurales/predict_6D/ ../images_s2/

### Setup


In [10]:
PATHS = [#'../images_s2/train_6D/2021-02-01_2021-02-28/*.tif',
         # '../images_s2/predict_6D/2021-02-01_2021-02-28/*.tif',
        fullpath(f"{IMAGES_OUTPUT}/predict_6D/{DATA_DATE}/*.tif")
         ]
PATHS

['/app/images_s2/predict_6D/2021-02-01_2021-02-28/*.tif']

### Compute  NDSW index


Generamos la banda Normalized Difference ShortWave (NDSW) como la diferencia normalizada entre las dos bandas del infrarrojo de onda corta, es decir NDSW = (SWIR12-SWIR11)/(SWIR12 + SWIR11 ) . Esta diferencia de bandas mostró ser sensible a la emisión de metano en estudios previos.  Dicho componente puede ser un indicador de la presencia de un basural con residuos biológicos.


In [17]:
for path in PATHS:
   
    images = glob.glob(path) #ruta a cada imagen 
    
    for image in images:
        
        #skip resulting files 
        if "_diffSW" in image: continue
        if "_ori_tmp" in image: continue
        if "rgb-nir-sw-diffSW"  in image: continue
            
        fname, ext = os.path.splitext(image)
        
        #--------------------------------
        #  Generacion de la banda NDSW  
        #--------------------------------
        
        ndsw_file = str(fname+'_diffSW.tif')
        print(ndsw_file)
        print("rgb")
        rgb_nir_swir_diffSW_fname = str(fname+'_rgb-nir-sw-diffSW.tif')
        print(rgb_nir_swir_diffSW_fname)

/app/images_s2/predict_6D/2021-02-01_2021-02-28/0000000000-0000013568_diffSW.tif
rgb
/app/images_s2/predict_6D/2021-02-01_2021-02-28/0000000000-0000013568_rgb-nir-sw-diffSW.tif
/app/images_s2/predict_6D/2021-02-01_2021-02-28/0000000000-0000000000_diffSW.tif
rgb
/app/images_s2/predict_6D/2021-02-01_2021-02-28/0000000000-0000000000_rgb-nir-sw-diffSW.tif


In [ ]:
print(path_to_files)

In [11]:

# para cada imagen en los grupos de entrenamiento prediccion y control, generamos la banda NDSW
for path in PATHS:
   
    images = glob.glob(path) #ruta a cada imagen 
    
    for image in tqdm(images):
        
        #skip resulting files 
        if "_diffSW" in image: continue
        if "_ori_tmp" in image: continue
        if "rgb-nir-sw-diffSW"  in image: continue
            
        fname, ext = os.path.splitext(image)
        
        #--------------------------------
        #  Generacion de la banda NDSW  
        #--------------------------------
        
        ndsw_file = str(fname+'_diffSW.tif')
        #usamos gdal_calc para calcular la nueva banda usando la banda 5 y 6 de cada imagen
        !gdal_calc.py -A $image --A_band=5 -B $image --B_band=6 --calc="(A-B)/(A+B)" --outfile=$ndsw_file 
        
        #------------------------------------------
        #  Generacion de la bandas RGB - NIR -SWIR  
        #------------------------------------------
        
        img_5bands = str(fname+'_ori_tmp.tif')
        #usamos gdal_translate quedarnos con las 5 primeras bandas de cada imagen
        !gdal_translate -b 1 -b 2 -b 3 -b 4 -b 5 $image $img_5bands

        #----------------------------------------------
        #  Generacion de imagen RGB - NIR -SWIR -NDSW 
        #----------------------------------------------        
        # FIX:el output sigue siendo el dir donde se descargaron las imagenes de google originalmente
        rgb_nir_swir_diffSW_fname = str(fname+'_rgb-nir-sw-diffSW.tif')
        #usamos otbcli_ConcatenateImages para concatenar las imagenes generadas en el paso anterior quedarnos con las 5 primeras bandas de cada imagen
        # TODO revieb profile
        # !bash -c "source /opt/OTB/otbenv.profile; otbcli_ConcatenateImages -il  $img_5bands $ndsw_file -out $rgb_nir_swir_diffSW_fname"
        !bash -c "otbcli_ConcatenateImages -il  $img_5bands $ndsw_file -out $rgb_nir_swir_diffSW_fname"


  0%|                                                                   | 0/2 [00:00<?, ?it/s]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 

 50%|█████████████████████████████                             | 1/2 [01:52<01:52, 112.03s/it]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 

100%|██████████████████████████████████████████████████████████| 2/2 [03:20<00:00, 100.22s/it]


## Generación del dataset de entrenamiento

El modelo de ML utiliza para entrenar un dataset compuesto por imágenes y máscaras binarias que delimitan el objeto de interés. Estas se generan vía la herramienta **satproc_extract_chips**. Para el caso del dataset de entrenamiento se generan imágenes y máscaras (utilizando las anotaciones de verdad de campo) y para el de predicción solo imágenes.

Para generar las imágenes de entrenamiento:

In [12]:
# not needed
# !pip install pysatproc

In [18]:
# FIX: es posible que el proceso anterior no deja las imagenes aca?
# path_to_files = fullpath(f"data/img/img_train/{DATA_DATE}/*_rgb-nir-sw-diffSW.tif")
path_to_files = fullpath(f"{IMAGES_OUTPUT}/predict_6D/{DATA_DATE}/*_rgb-nir-sw-diffSW.tif")
print("Path to files: ", path_to_files)
output_folder = fullpath("dataset/data_train/6D/100_30/")
print("Output Folder: ", output_folder)
vector_file = fullpath("data/shp/gt/gt_basurales_4326.geojson")

size = 100
step_size = 30

Path to files:  /app/images_s2/predict_6D/2021-02-01_2021-02-28/*_rgb-nir-sw-diffSW.tif
Output Folder:  /app/dataset/data_train/6D/100_30


In [19]:
!satproc_extract_chips \
    $path_to_files \
    -o  $output_folder \
    --size $size \
    --step-size $step_size \
    --aoi $vector_file \
    --labels $vector_file \
    --label-property 'class' \
    --classes 'B' \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100

Rasters:   0%|                                            | 0/2 [00:00<?, ?it/s]
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows: 0it [00:00, ?it/s]
Rasters:  50%|##################                  | 1/2 [00:03<00:03,  3.63s/it]
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:   0%| | 0/61 [00:00<?, ?it/
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:   3%| | 2/61 [00:00<00:03, 
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:  16%|1| 10/61 [00:00<00:01,
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:  31%|3| 19/61 [00:00<00:00,
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:  43%|4| 26/61 [00:00<00:00,
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:  56%|5| 34/61 [00:00<00:00,
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows:  75%|7| 46/61 [00:00<00:00,
0000000000-0000013568_rgb-nir-sw-diffSW.tif windows: 100%|#| 61/61 [00:00<00:00,
Rasters: 100%|####################################| 2/2 [00:08<00:00,  4.31s/it]


###### Los argumentos:

* **El primer argumento** es la ruta a las imágenes 

* **o** es la ruta de destino 

Recomendamos que dicha ruta sea descriptiva, por ejemplo “data_train/600_600/ ” describe : Data_train → datos usados para entrenar; 400_400 → <tamaño de la imagen >_ <tamaño del step-size> (las imágenes son cuadradas)

* **size** tamaño de las imágenes resultantes (las imágenes son cuadradas) 
* **step-size** paso del proceso. Si *step-size* es igual que el *size* entonces no hay overlap en las imágenes. 

En ocaciones es útil para el entrenamiento generar los chips con un overlap de este modo tenemos más datos para entrenar. Pero en la predicción valor debe ser igual al tamaño que la imagen 

* **crs** epsg: le asigna un epsg a la imagen 

* **label-property** nombre del campo donde se define cada categoría (solo se usa para el entrenamiento) 

* **classes** nombres de las clases (como aparecen en el geojson), separados por espacios

* **aoi** ruta al archivo vectorial donde están definidas las localidades. Al definir una region de interés solo se procesan las imágenes que interceptan esas localidades.

* **rescale** lleva los valores de las bandas a 0-255

Este comando va a generar dos carpetas en la ruta de destino : “images” y “masks”. Los archivos de la primera van a ser de tipo Tiff de 3 bandas (rgb) y los de la segunda van a ser, también, de tipo Tiff pero de N bandas donde N representa el número de clases, en este caso sólo una. Y donde cada una de las bandas es una máscara binaria


## Generación del dataset de predicción


De modo similar repetimos el procedimiento. En este caso el unico archivo vectorial delimita las zonas de interés para la predicción. 

### dataset de predicción

In [2]:
#!satproc_extract_chips --help

In [6]:
!ls ../dataset/data_predict/6D/100_100/

images


In [21]:
# path_to_files = "../images_s2/predict_6D/2021-02-01_2021-02-28/*_rgb-nir-sw-diffSW.tif"
path_to_files = fullpath(f"{IMAGES_OUTPUT}/predict_6D/{DATA_DATE}/*_rgb-nir-sw-diffSW.tif")
# output_folder = "../dataset/data_predict/6D/100_100/"
output_folder = fullpath("dataset/data_predict/6D/100_100/")
mkdir(output_folder)

mkdir(fullpath("data/shp/"))
vector_file_aoi = fullpath("data/shp/aoi-predict.geojson")

size = 100


!satproc_extract_chips \
    $path_to_files \
    -o $output_folder \
    --size $size \
    --step-size $size \
    --aoi $vector_file_aoi \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100 \
    --skip-existing

Rasters:   0%|                                            | 0/2 [00:00<?, ?it/s]
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   0%| | 0/18496 [00:00<?, ?
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   0%| | 29/18496 [00:00<01:
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   0%| | 65/18496 [00:00<00:
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 100/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 135/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 169/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 202/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 236/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 271/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   2%| | 307/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   2%| | 343/18496 [00:01<00
0000000000-0000000000_rgb-ni